In [1]:
import os
os.getcwd() 

'/Users/izapreev/Projects/ML-PT'

In [2]:
# Make sure the source code auto reloads into the kernel\n
%load_ext autoreload
%autoreload 2

In [3]:
from src.utils.logger import logger

In [6]:
from src.utils.file_utils import load_numpy_zc

# Load the features file for further use
file_name = os.path.join('.', 'data', 'X.npz')
X = load_numpy_zc(file_name, ['X'])

15:26:55 INFO (file_utils:32): Loading compressed numpy z file from: ./data/X.npz
15:26:55 INFO (file_utils:35): File loading and data extraction are done!


In [ ]:
# TODO: Apply DBSCAN for clustering of the provided data (tune hyperparameters to find the optimal number of clusters)

In [ ]:
# TODO: For each cluster compute the central point and the radius

In [ ]:
# TODO: Store the trained models (TfidfVectorizer, PCA and etc) along with the Classifier information: Class Id, Central point, Radius